
    CREATE TABLE IF NOT EXISTS code_completion 
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        note_id INTEGER,
        dataframe_header JSON,
        solution_dataframe JSON,
        problem_description TEXT,
        dataset_path TEXT,
        FOREIGN KEY (note_id) REFERENCES notes (id)           


In [1]:
import sqlite3

def write_example_to_db(dataframe_head, dataframe_target, dataset_path, description):
    conn = sqlite3.connect("../../flashcards.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO notes DEFAULT VALUES")
    note_id = cursor.lastrowid
    cursor.execute(
        "INSERT INTO code_completion (note_id, dataframe_header, solution_dataframe, problem_description, dataset_path) VALUES (?, ?, ?, ?, ?)",
        (note_id, dataframe_head.to_json(), dataframe_target.to_json(), description, dataset_path),
    )
    conn.commit()
    conn.close()


In [2]:
import polars as pl
import pandas as pd 


dataset_path = "backend/code_completion/data/contoso_sales.csv"

df = pl.read_csv("../../"+dataset_path)

# transformation
result = (
    df
    .group_by('Brand')
    .agg(pl.col('Quantity').sum().alias('Sum of Quantity'))
)

input_header = pd.DataFrame(df.head(3).to_dict())

result = pd.DataFrame(result.to_dict())

description = "Summarise the brand column by taking the sum of Quanaity, naming the new column 'Sum of Quanity'"
# 
write_example_to_db(input_header, result, dataset_path, description)